# Contrôle complet de l'entraînement et d'Optuna

Ce notebook démontre comment modifier **tous** les paramètres du pipeline PhysAE : données, modèle, entraînement par stages et recherches Optuna.

## Imports

On rassemble ici toutes les dépendances nécessaires pour instancier le pipeline et lancer des optimisations.

In [ ]:
from copy import deepcopy
from pprint import pprint

import optuna
import pytorch_lightning as pl
import torch

from physae import build_data_and_model, optimise_stage, train_stage_custom
from physae.config_loader import load_data_config, load_stage_config, merge_dicts

## Inspection des configurations de base

Les fichiers YAML fournis servent de point de départ. On peut les afficher et les modifier librement en mémoire.

In [ ]:
data_cfg = load_data_config(name='default')
stage_A_cfg = load_stage_config('A')
print('Résumé data:')
print({k: data_cfg[k] for k in ['n_points', 'n_train', 'n_val', 'batch_size']})
print('
Hyperparamètres du stage A:')
pprint({k: stage_A_cfg[k] for k in ['epochs', 'base_lr', 'refiner_lr', 'train_base', 'train_heads']})

## Surcharges complètes des données et du modèle

On peut ajuster le moindre champ : tailles, plages physiques, bruit, architecture du réseau et optimiseur. Cette cellule crée un nouveau lot de loaders et un modèle configuré avec ces modifications.

In [ ]:
data_overrides = {
    'n_train': 2048,
    'n_val': 256,
    'batch_size': 32,
    'train_ranges': {
        'sig0': [3085.42, 3085.47],
        'dsig': [0.0015, 0.0016],
        'mf_CH4': [3e-6, 5e-5],
        'baseline0': [0.98, 1.02],
        'baseline1': [-6e-4, -1.5e-4],
        'baseline2': [-6e-8, -1.5e-8],
        'P': [350.0, 700.0],
        'T': [295.0, 330.0],
    },
    'val_ranges': {
        'sig0': [3085.43, 3085.46],
        'dsig': [0.00152, 0.00156],
        'mf_CH4': [5e-6, 3e-5],
        'baseline0': [0.99, 1.01],
        'baseline1': [-4e-4, -2.5e-4],
        'baseline2': [-4.5e-8, -2.5e-8],
        'P': [380.0, 620.0],
        'T': [300.0, 315.0],
    },
    'noise': {
        'train': {
            'std_add_range': [0.0, 0.02],
            'std_mult_range': [0.0, 0.015],
            'p_drift': 0.4,
            'drift_sigma_range': [10.0, 80.0],
            'drift_amp_range': [0.003, 0.05],
            'p_fringes': 0.4,
            'fringe_freq_range': [0.3, 25.0],
            'fringe_amp_range': [0.001, 0.02],
            'p_spikes': 0.2,
            'spikes_count_range': [1, 5],
            'spike_amp_range': [0.001, 0.4],
            'spike_width_range': [1.0, 12.0],
            'clip': [0.0, 1.25],
        },
        'val': {
            'std_add_range': [0.0, 5e-4],
            'std_mult_range': [0.0, 5e-4],
            'p_drift': 0.0,
            'drift_sigma_range': [15.0, 120.0],
            'drift_amp_range': [0.0, 0.015],
            'p_fringes': 0.0,
            'fringe_freq_range': [0.5, 12.0],
            'fringe_amp_range': [0.0, 0.005],
            'p_spikes': 0.0,
            'spikes_count_range': [1, 2],
            'spike_amp_range': [0.0, 0.02],
            'spike_width_range': [1.0, 3.0],
            'clip': [0.0, 1.1],
        },
    },
    'predict_list': ['sig0', 'dsig', 'mf_CH4', 'P', 'T', 'baseline1', 'baseline2'],
    'film_list': ['sig0', 'P', 'T'],
    'lrs': [3e-4, 1e-4],
    'model': {
        'encoder': {
            'name': 'efficientnet',
            'params': {
                'width_mult': 1.25,
                'depth_mult': 1.1,
                'expand_ratio_scale': 1.1,
                'se_ratio': 0.3,
                'norm_groups': 8,
            },
        },
        'shared_head_hidden_scale': 0.6,
        'refiner': {
            'name': 'efficientnet',
            'params': {
                'width_mult': 0.9,
                'depth_mult': 1.1,
                'expand_ratio_scale': 1.05,
                'se_ratio': 0.3,
                'norm_groups': 8,
                'hidden_scale': 0.55,
            },
        },
        'optimizer': {
            'name': 'adamw',
            'betas': [0.92, 0.999],
            'weight_decay': 5e-5,
        },
        'scheduler': {
            'eta_min': 1e-8,
            'T_max': 200,
        },
    },
}

model, (train_loader, val_loader), metadata = build_data_and_model(config_overrides=data_overrides)
print('Tailles des loaders :', len(train_loader.dataset), len(val_loader.dataset))
print('Batch size :', train_loader.batch_size)
print('Paramètres à prédire :', metadata['predict_list'])

## Paramétrage fin du stage d'entraînement

La fonction `train_stage_custom` accepte directement des surcharges pour chaque hyperparamètre : l'exemple ci-dessous illustre comment activer/désactiver des blocs, changer d'optimiseur ou ajuster le scheduler.

In [ ]:
stage_overrides = {
    'stage_name': 'A',
    'epochs': 8,
    'base_lr': 3e-4,
    'refiner_lr': 1e-4,
    'train_base': True,
    'train_heads': True,
    'train_film': True,
    'train_refiner': True,
    'refine_steps': 1,
    'delta_scale': 0.12,
    'use_film': True,
    'film_subset': ['sig0', 'P'],
    'heads_subset': ['sig0', 'dsig', 'mf_CH4'],
    'baseline_fix_enable': True,
    'optimizer': 'adamw',
    'optimizer_weight_decay': 5e-5,
    'optimizer_beta1': 0.93,
    'optimizer_beta2': 0.9993,
    'scheduler_eta_min': 5e-8,
    'scheduler_T_max': 120,
    'accelerator': 'cpu',
    'enable_progress_bar': True,
    'trainer_kwargs': {
        'gradient_clip_val': 1.0,
        'accumulate_grad_batches': 2,
        'precision': 32,
    },
}

# Exemple d'appel (désactivé par défaut pour éviter un entraînement long)
# train_stage_custom(model, train_loader, val_loader, **stage_overrides)
print('Exemple de configuration prête pour train_stage_custom :')
pprint(stage_overrides)

## Création d'un espace de recherche Optuna sur mesure

On peut enrichir les espaces `optuna` en mémoire avant d'appeler `optimise_stage`. Ici on optimise à la fois des hyperparamètres du stage et des plages de données.

In [ ]:
stage_A_search = deepcopy(stage_A_cfg)
stage_A_search['optuna'].update({
    'epochs': {'type': 'int', 'low': 6, 'high': 18},
    'base_lr': {'type': 'float', 'low': 5e-5, 'high': 8e-4, 'log': True},
    'optimizer_weight_decay': {'type': 'float', 'low': 1e-6, 'high': 1e-4, 'log': True},
    'data.train_ranges.mf_CH4.low': {'type': 'float', 'low': 2e-6, 'high': 1e-5, 'log': True},
    'data.train_ranges.mf_CH4.high': {'type': 'float', 'low': 2e-5, 'high': 8e-5, 'log': True},
    'data.noise.train.std_add_range.high': {'type': 'float', 'low': 0.005, 'high': 0.03},
})

print('Espace de recherche Optuna enrichi :')
pprint(stage_A_search['optuna'])

## Lancement d'une optimisation Optuna complète

Le bloc suivant montre comment lancer un petit nombre d'essais tout en contrôlant sampler, pruner et répertoires d'artefacts. Adapter `n_trials` et les plages selon vos besoins.

In [ ]:
sampler = optuna.samplers.TPESampler(seed=123)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=1)

study = optimise_stage(
    'A',
    n_trials=3,
    metric='val_loss',
    direction='minimize',
    data_config_name='default',
    data_overrides={'n_train': 1024, 'n_val': 128},
    stage_overrides={'epochs': 6},
    sampler=sampler,
    pruner=pruner,
    output_dir='artifacts/optuna_demo',
    save_figures=False,
)

study.trials_dataframe()